## Data preparing

In [ ]:
## Example from intents.json


#tag: label for input
#patterns: sentence for classification in training procedure
#responses: bot's replies 

# {"intents": [
#         {"tag": "greeting",
#          "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],
#          "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],
#          "context_set": ""
#         },
#         {"tag": "goodbye",
#          "patterns": ["Bye", "See you later", "Goodbye"],
#          "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]
#         },
#         {"tag": "thanks",
#          "patterns": ["Thanks", "Thank you", "That's helpful"],
#          "responses": ["Happy to help!", "Any time!", "My pleasure"]
#         },
#         {"tag": "hours",
#          "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],
#          "responses": ["We're open every day 9am-9pm", "Our hours are 9am-9pm every day"]
#         },
#         {"tag": "menu",
#          "patterns": ["I want to order", "What kinds of food are there?", "What is your delicous food?" ],
#          "responses": ["Are you ready to order? Our restaurant have pizza, vegetable pasta, beefsteack. These is special of our restaurant", "Can I help you? This is menu. Please ask me if you have any question."]
#         }
#     ]
# }

## Training model

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
 
import numpy as np
import tflearn
import tensorflow as tf
import random

In [2]:
#loading data
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
#pre-processing data
 
words = [] # a list of all words
classes = [] # like tags
documents = [] # list( words and their corresponding tag)
stop_words = ['?', 'a', 'an', 'the']
 
for intent in intents['intents']:
    for pattern in intent['patterns']:
 
        w = nltk.word_tokenize(pattern)
        words.extend(w) #add w to words
        documents.append((w, intent['tag'])) #add word and corresponding tag
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
 
words = [stemmer.stem(w.lower()) for w in words if w not in stop_words]
words = sorted(list(set(words)))
 
classes = sorted(list(set(classes)))

In [13]:
print(words)
print(classes)
print(documents)

["'s", 'anyon', 'ar', 'bye', 'day', 'del', 'food', 'good', 'goodby', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'of', 'op', 'ord', 'see', 'thank', 'that', 'ther', 'to', 'want', 'what', 'when', 'yo', 'you']
['goodbye', 'greeting', 'hours', 'menu', 'thanks']
[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['I', 'want', 'to', 'order'], 'menu'), (['What', 'kinds', 'of', 'food', 'are', 'there', '?'], 'menu'), (['What', 'is', 'your', 'delicous', 'food', '?'], 'menu')]


In [22]:
# create our training datad
training = []
output = []
output_empty = [0] * len(classes)
# print(output_empty) # [0, 0, 0, 0, 0]

# training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0] #get the first part in each document, containing only words; doc[1] is tag
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    print(pattern_words)
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    print(bag)
    
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    print(output_row)
    
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)
print(training)


# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])
 

['hi']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
['how', 'ar', 'you']
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0]
['is', 'anyon', 'ther', '?']
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
['hello']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
['good', 'day']
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
['bye']
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
['see', 'you', 'lat']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0]
['goodby']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0,

In [12]:
print(pattern_words)
print(words)
print(bag)
print(output_row)

['what', 'is', 'yo', 'del', 'food', '?']
["'s", 'anyon', 'ar', 'bye', 'day', 'del', 'food', 'good', 'goodby', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'of', 'op', 'ord', 'see', 'thank', 'that', 'ther', 'to', 'want', 'what', 'when', 'yo', 'you']
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 0]


In [23]:
print(train_x)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [24]:
print(train_y)

[[0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0]]


## Start training

In [41]:
 
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8) #build 2-layer NN for traiing
# net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')
 
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

Training Step: 2999  | total loss: 0.00194 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00194 - acc: 1.0000 -- iter: 16/17
Training Step: 3000  | total loss: 0.00189 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.00189 - acc: 1.0000 -- iter: 17/17
--
INFO:tensorflow:/home/hoang/Documents/svn/reposn/code/chatbot/models/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [42]:
#Save the model to reuse later (optional)
 
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )
 

## Predict (build chatbot)

In [43]:
# restore training data
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
 
# relad jsondata to get responses
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [44]:
# Load tensorflow model
model.load('models/model.tflearn')

INFO:tensorflow:Restoring parameters from /home/hoang/Documents/svn/reposn/code/chatbot/models/model.tflearn


## Process user input
We tokenize, convert user input to BOW and classify later 

In [53]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
 
# bag of words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    print(sentence_words)
    # bag of words
    bag = [0]*len(words) 
#     print(bag)#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
 
    return(np.array(bag))

In [57]:
bow('I want some special food', words)

['i', 'want', 'som', 'spec', 'food']


array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0])

In [92]:
# data structure to hold user context
context = {}
 
ERROR_THRESHOLD = 0.25

#to evaluate proportion that a sentence belongs to a certain class
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0] #predict the sentence with the pre-processed words earlier
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
#     print(results)
    results.sort(key=lambda x: x[1], reverse=True) #sort in score desc.
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

#give a potential response of request
def response(sentence, userID='1', show_details=False):
    results = classify(sentence)
    print(results)
    if results:
        while results:
            for intent in intents['intents']:
                if intent['tag'] == results[0][0]:
                    if 'context_set' in intent:
                        if show_details: print ('context:', intent['context_set'])
                        context[userID] = intent['context_set']
                    if not 'context_filter' in intent or \
                        (userID in context and 'context_filter' in intent and intent['context_filter'] == context[userID]):
                        if show_details: print ('tag:', intent['tag'])
                        return print(random.choice(intent['responses']))
 
            results.pop(0)
 

## Testing

In [97]:
response('Hello!')

['hello', '!']
[('greeting', 0.99645871)]
Hi there, how can I help?
